In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tf.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# # 预处理数据
# x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.
# x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.

# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)

learning_rate = 1e-4
keep_prob_rate = 0.7 # 
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding= 'same')

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='same')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##

W_conv1 = weight_variable([7, 7, 1, 32])         # patch 7x7, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)   # 卷积  自己选择 选择激活函数
h_pool1 = max_pool_2x2(h_conv1) # 池化               

# 卷积层 2
W_conv2 = weight_variable([5, 5, 32, 64])   # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)   # 卷积  自己选择 选择激活函数
h_pool2 = max_pool_2x2(h_conv2) # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # idx = np.random.choice(x_train.shape[0], 100)
        # batch_xs = x_train[idx]
        # batch_ys = y_train[idx]
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
        if i % 100 == 0:
            print(compute_accuracy(
                mnist.test.images[:1000], mnist.test.labels[:1000]))


ModuleNotFoundError: No module named 'tf'

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


In [2]:
# 加载数据
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]  # (batch, 28, 28, 1)
x_test  = x_test[..., tf.newaxis]

y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

In [3]:
model = models.Sequential([
    layers.Conv2D(32, (7, 7), activation='relu', padding='same', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2), padding='same'),
    
    layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),  # keep_prob = 0.7 <=> dropout rate = 0.3
    layers.Dense(10, activation='softmax')
])

d:\anna\envs\py39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# 模型编译
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 模型训练
model.fit(x_train, y_train, batch_size=100, epochs=20, validation_split=0.1)

# 模型评估
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.7899 - loss: 0.8049 - val_accuracy: 0.9717 - val_loss: 0.1049
Epoch 2/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.9663 - loss: 0.1137 - val_accuracy: 0.9822 - val_loss: 0.0648
Epoch 3/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9770 - loss: 0.0722 - val_accuracy: 0.9835 - val_loss: 0.0587
Epoch 4/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9821 - loss: 0.0584 - val_accuracy: 0.9865 - val_loss: 0.0466
Epoch 5/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9865 - loss: 0.0469 - val_accuracy: 0.9875 - val_loss: 0.0414
Epoch 6/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.9883 - loss: 0.0381 - val_accuracy: 0.9880 - val_loss: 0.0398
Epoch 7/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9895 - loss: 0.0346 - val_accuracy: 0.9895 - val_loss: 0.0371
Epoch 8/20
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.9906 - loss: 0.0290 - 